In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, RandomForestRegressor

In [11]:
df_join = pd.read_excel('../data/data_joined_16-23_no_new_buildings.xlsx')

In [12]:
df_infl = pd.read_excel('../data/constructed/inflation-rate-in-kazakhstan-2028.xlsx', sheet_name='Data')
df_infl['Year'] = pd.to_numeric(df_infl['Year'].str.replace('*',''))
df_infl=df_infl[pd.to_numeric(df_infl.Year)>=2016]

C:\Users\User\AppData\Local\Temp\ipykernel_20876\1026236143.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_infl['Year'] = pd.to_numeric(df_infl['Year'].str.replace('*',''))


In [13]:
sample_means = df_join.groupby('YEAR', as_index=False).apply(lambda x: x.sample(20)).groupby('YEAR')['price_adj_per_area'].mean()


Validation of overall model

In [14]:
tree = DecisionTreeRegressor(criterion='absolute_error', min_samples_split=5, max_leaf_nodes=16)

In [15]:
scores=[]
kf = KFold(n_splits=10)
for year in df_join.YEAR.unique():
            
    x, y = df_join[df_join['YEAR']==year][[ 'floor', 'building_age', 'total_floor', 'Latitude', 'Longitude','Regions_Alatau', 'Regions_Almaly', 'Regions_Almaty', 'Regions_Auezov',
        'Regions_Bostandyq', 'Regions_Jetysu', 'Regions_Medeu',
        'Regions_Nauryzbai', 'Regions_Turksib','YEAR']], df_join[df_join['YEAR']==year]['price_adj_per_area']
    kf.get_n_splits(x, y)
    for train_index,test_index in kf.split(x, y):    
        X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        tree.fit(X_train, y_train)
        scores.append(tree.score(X_test, y_test))

In [16]:
np.array(scores).mean()

-1.9930321345202373

In [ ]:
tree = DecisionTreeRegressor(criterion='absolute_error', min_samples_split=5, max_leaf_nodes=16)

In [ ]:
scores=[]
kf = KFold(n_splits=10)
for year in df_join.YEAR.unique():
            
    x, y = df_join[df_join['YEAR']==year][[ 'floor', 'building_age', 'total_floor', 'Latitude', 'Longitude','Regions_Alatau', 'Regions_Almaly', 'Regions_Almaty', 'Regions_Auezov',
        'Regions_Bostandyq', 'Regions_Jetysu', 'Regions_Medeu',
        'Regions_Nauryzbai', 'Regions_Turksib','YEAR']], df_join[df_join['YEAR']==year]['price_adj_per_area']
    kf.get_n_splits(x, y)
    for train_index,test_index in kf.split(x, y):    
        X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        tree.fit(X_train, y_train)
        scores.append(tree.score(X_test, y_test))